In [1]:
import datetime
import pandas as pd
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import re

from fuzzywuzzy import process, fuzz
from notnews import *

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Bashar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/Bashar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
FILE_PATH = '../dataverse_files/'
PROCESSED_PATH = FILE_PATH + 'processed/'

# Processing files

In [3]:
full_df = pd.DataFrame()

## Processing CNN & NBC News File

In [4]:
news_df = pd.read_csv(PROCESSED_PATH + 'cnn_nbc_news_soft_news_predict.csv', encoding = 'ISO-8859-1', low_memory=False)

In [5]:
news_df.loc[news_df['url'].str.contains('cnn.com'),'channel.name'] = 'CNN'
news_df.loc[news_df['url'].str.contains('nbcnews.com'),'channel.name'] = 'NBC News'

In [6]:
# Default to CNN and then get the index prior to splitting into mulit-records and assign all records
#  to CNN International if the transcript contains CNN International in its text.
mod_list = news_df.loc[(news_df['channel.name'] == 'CNN') & (news_df['text'].str.contains('CNN INTERNATIONAL', case=False))]['Unnamed: 0'].tolist()    
news_df.iloc[mod_list,2] = 'CNN International'

In [7]:
# delete any outliers
news_df.drop(news_df[news_df['year'] > 2100.0].index , axis=0, inplace=True)
news_df.drop(news_df[news_df['year'] < 1900.0].index , axis=0, inplace=True)    

# generate date field in format YYYY-MM-DD
news_df['full_date'] = pd.to_datetime((news_df.year*10000+news_df.month*100+news_df.date),format='%Y%m%d')

# cleaning up the dataframe and appending to the full dataframe
news_df = news_df[news_df.columns[news_df.columns.isin(['channel.name','program.name','full_date','prob_soft_news_us','text'])]]
full_df = pd.concat([full_df, news_df], axis=0)


In [8]:
del news_df

## Processing MSNBC files

In [9]:
news_df = pd.read_csv(PROCESSED_PATH + 'msnbc--2003--2014_soft_news_predict.csv')
news_df.rename(columns={'Source':'channel.name', 'Show':'program.name', 'Content':'text'},inplace=True)
news_df['full_date'] = pd.to_datetime(news_df.Date)

# keeping only the required columns in the dataframe
news_df = news_df[news_df.columns[news_df.columns.isin(['channel.name','program.name','full_date','prob_soft_news_us','text'])]]

In [10]:
news2_df = pd.read_csv('../dataverse_files/processed/soft_news_msnbc-2010--2021.csv')
news2_df.drop('program.name', axis=1, inplace=True)
news2_df.rename(columns={'Source':'channel.name', 'show_name':'program.name'},inplace=True)
news2_df['full_date'] = pd.to_datetime(news2_df.air_date)

# keeping only the required columns in the dataframe
news2_df = news2_df[news2_df.columns[news2_df.columns.isin(['channel.name','program.name','full_date','prob_soft_news_us','text'])]]

/Users/Bashar/opt/anaconda3/envs/nonconform/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [11]:
print (f'Size of 2003-2014 df: {news_df.shape}')
print (f'Size of 2010-2021 df: {news2_df.shape}')

Size of 2003-2014 df: (252433, 5)
Size of 2010-2021 df: (163103, 5)


Merging the 2 dataframes to see if there is any redundant records

In [12]:
news_df.merge(news2_df, how='outer', on=['channel.name','program.name','full_date', 'prob_soft_news_us','text'])

,channel.name,program.name,text,prob_soft_news_us,full_date
0,MSNBC,HARDBALL,"b'CHRIS MATTHEWS, HOST: I`m Chris Matthews. Le...",0.117768,2002-09-24
1,MSNBC,HARDBALL,difference between the president`s speech at t...,0.048173,2002-09-24
2,MSNBC,HARDBALL,it`s a combination of worries about the lack o...,0.485402,2002-09-24
3,MSNBC,HARDBALL,going on in the war with Iraq. Congressman Rob...,0.086091,2002-09-24
4,MSNBC,HARDBALL,"it`s\ngoing to be. I mean, I think the preside...",0.265826,2002-09-24
...,...,...,...,...,...
415531,MSNBC,The 11th Hour,about another recent poll that finds wide part...,0.058611,2022-02-02
415532,MSNBC,The 11th Hour,"effective. (END VIDEO CLIP) JANSING: Tonight, ...",0.086091,2022-02-02
415533,MSNBC,The 11th Hour,provide a strong enough immune response in kid...,0.086091,2022-02-02
415534,MSNBC,The 11th Hour,"is the case, it might well be the case. But, y...",0.321647,2022-02-02


In [13]:
full_df = pd.concat([full_df, news_df], axis=0)
full_df.shape

(3094284, 5)

In [14]:
# fixing the records with wrong channel name
full_df.loc[full_df['channel.name'] == 'MSNBC ~~FDCH', 'channel.name'] = 'MSNBC'

In [15]:
del news_df
del news2_df

# Cleaning Data

In [16]:
# dropping any record that doesn't have a program name associated to it
full_df.drop(full_df.loc[full_df['program.name'].isna()].index, axis=0, inplace=True)
full_df.shape

(3081731, 5)

In [17]:
def get_matching_shows(df):
    unique_shows = df['program.name'].unique().tolist()
    
    # creating a tuple with each unique show and its closest matches with FuzzyWuzzy token sort ration method
    match_tuple = [(x,) + i 
                   for x in unique_shows 
                   for i in process.extract(x,unique_shows, scorer=fuzz.token_sort_ratio)]

    analysis_df = pd.DataFrame(columns=['program.name','matched.program', 'score'], data=match_tuple)
    
    return analysis_df

In [18]:
matching_show_df = get_matching_shows(full_df)

# filter for shows that only match > 70
matching_show_df = matching_show_df[(matching_show_df.score > 80) & (matching_show_df.score < 100)]
matching_show_df.to_csv(FILE_PATH + 'raw_matching_shows.csv', index=False)

After exporting the matching program names with fuzzy wuzzy, manually went through the file and only kept one show to replace each program. If a show listed was not to be changed then the entry was ommitted. This was used as a lookup in the code below to do most of the heavy lifting of converting shows into the same string. After that we still had some that had to be edited manaually

In [19]:
matching_show_df = pd.read_csv(FILE_PATH + 'show_lookup.csv')
#matching_show_df.reset_index(drop=True)
#matching_show_df.drop(['Unnamed: 0', 'score'],axis=1, inplace=True)
matching_show_df

,program.name,matched.program
0,CNN International Best Of Q&A;,CNN INTERNATIONAL Q&A
1,CNN International Best Of Insight,CNN INTERNATIONAL INSIGHT
2,CNN AHEAD OF THE CURVE,Ahead Of The Curve
3,CNN BREAKING NEWS,Breaking News
4,CNN BURDEN OF PROOF,Burden of Proof
...,...,...
77,Hardball with Chris Matthews' forWednesday,Hardball with Chris Matthews
78,The Rachel Maddow Show' Friday,The Rachel Maddow Show
79,The Last Word with Lawrence O'Donnell' Wednesday,The Last Word with Lawrence O'Donnell
80,The Last Word with Lawrence O'Donnell' forThur...,The Last Word with Lawrence O'Donnell


In [20]:
full_df.loc[full_df['program.name'].isin(matching_show_df['program.name']),['program.name']] = matching_show_df['matched.program']

In [21]:
full_df['program.name'] = full_df['program.name'].str.title()
full_df.drop(full_df[full_df['program.name'].isnull()].index, axis=0, inplace=True)
sorted(full_df['program.name'].unique().tolist())

['@Thishour With Berman And Michaela',
 'Ac 360 Degrees',
 'Ac 360 Later',
 'Ahead Of The Curve',
 'All In With Chris Hayes',
 'Amanpour',
 'America`S Voices',
 'American Morning',
 'American Morning: Wake Up Call',
 'American Nightly Scoreboard',
 'Anderson Cooper 360 Degrees',
 'Anthony Bourdain Parts Unknown',
 'Around The World',
 'Asia Tonight',
 'At This Hour',
 'Ballot Bowl 2008',
 'Beck',
 'Biz Asia',
 'Both Sides With Jesse Jackson',
 'Breaking News',
 'Buchanan & Press',
 'Burden Of Proof',
 'Business Traveler',
 'Business Unusual',
 'Business View',
 'Campbell Brown',
 'Campbell Brown: No Bias, No Bull',
 'Capital Gang',
 'Chicagoland',
 'Closing Bell',
 'Cnn 10',
 'Cnn America Strikes Back',
 'Cnn American Morning With Paula Zahn',
 'Cnn American Stories',
 'Cnn Christmas Around The World',
 'Cnn Connie Chung Tonight',
 'Cnn Daybreak',
 'Cnn Election Center',
 'Cnn Go',
 'Cnn He Said/She Said',
 'Cnn Hotspots',
 'Cnn In The Money',
 'Cnn Insight',
 'Cnn International Asia T

In [22]:
# Fixing the shows that fuzzy wuzzy didn't catch, the list was small enough that doing it by hand was a lot quicker
full_df.loc[full_df['program.name']=='Ac 360 Degrees','program.name'] = 'Anderson Cooper 360 Degrees'
full_df.loc[full_df['program.name']=='American Morning With Paula Zahn','program.name'] = 'American Morning'
full_df.loc[full_df['program.name']=='Cnn American Morning With Paula Zahn','program.name'] = 'American Morning'
full_df.loc[full_df['program.name']=='Cnn International Q&A;','program.name'] = 'Cnn International Q&A'
full_df.loc[full_df['program.name']=='Cnn Late Edition With Wolf Blitzer','program.name'] = 'Cnn Late Edition'
full_df.loc[full_df['program.name']=='Cnn News Night Aaron Brown','program.name'] = 'Cnn Newsnight With Aaron Brown'
full_df.loc[full_df['program.name']=='Cnn Newsnight Aaron Brown','program.name'] = 'Cnn Newsnight With Aaron Brown'
full_df.loc[full_df['program.name']=='Cnn Page One With Nick Charles','program.name'] = 'Cnn Page One'
full_df.loc[full_df['program.name']=='Cnn Saturday Edition','program.name'] = 'Cnn Saturday'
full_df.loc[full_df['program.name']=='Cnn Showdown On Iraq','program.name'] = 'Cnn Showdown: Iraq'
full_df.loc[full_df['program.name']=='Cnn The Point With Greta Van Susteren','program.name'] = 'Cnn The Point'
full_df.loc[full_df['program.name']=='Cnn The Spin Room Corrected Copy','program.name'] = 'Cnn The Spin Room'
full_df.loc[full_df['program.name']=="CNN'S AMANPOUR",'program.name'] = 'Amanpour'
full_df.loc[full_df['program.name']=='Cnn&Time;','program.name'] = 'CNN/Time'
full_df.loc[full_df['program.name']=='Evans, Novak, Hunt & Shields','program.name'] = 'Cnn Evans, Novak, Hunt & Shields'
full_df.loc[full_df['program.name']=='Hardball With Chris Matthews','program.name'] = 'Hardball'
full_df.loc[full_df['program.name']=="Hardball With Chris Matthews' Fortuesday",'program.name'] = 'Hardball'
full_df.loc[full_df['program.name']=="Hardball With Chris Matthews' Forwednesday",'program.name'] = 'Hardball'
full_df.loc[full_df['program.name']=='Jane Velez-Mitchell','program.name'] = 'Issues With Jane Velez-Mitchell'
full_df.loc[full_df['program.name']=="Judy Woodruffs'S Inside Politics",'program.name'] = "Judy Woodruff'S Inside Politics"
full_df.loc[full_df['program.name']=='Melissa-Harris-Perry','program.name'] = 'The Melissa Harris-Perry Show'
full_df.loc[full_df['program.name']=='Melissa Harris-Perry','program.name'] = 'The Melissa Harris-Perry Show'
full_df.loc[full_df['program.name']=='Msnbc Hardball','program.name'] = 'Hardball'
full_df.loc[full_df['program.name']=='Politicsnation','program.name'] = 'Politics Nation'
full_df.loc[full_df['program.name']=='The Ed Show With Ed Schultz','program.name'] = 'The Ed Show'
full_df.loc[full_df['program.name']=='The Ed Show Forthursday,July 19Th','program.name'] = 'The Ed Show'
full_df.loc[full_df['program.name']=="The Last Word With Lawrence O' Donnell",'program.name'] = "The Last Word With Lawrence O'Donnell"
full_df.loc[full_df['program.name']=="The Last Word With Lawrence O'Donnell' Forthursday",'program.name'] = "The Last Word With Lawrence O'Donnell"
full_df.loc[full_df['program.name']=="The Last Word With Lawrence O'Donnell' Fortuesday",'program.name'] = "The Last Word With Lawrence O'Donnell"
full_df.loc[full_df['program.name']=="The Last Word With Lawrence O'Donnell' Fothursday",'program.name'] = "The Last Word With Lawrence O'Donnell"
full_df.loc[full_df['program.name']=="The Last Word With Lawrence O'Donnell' Wednesday",'program.name'] = "The Last Word With Lawrence O'Donnell"
full_df.loc[full_df['program.name']=='The Point With Greta Van Susteren','program.name'] = 'The Point'
full_df.loc[full_df['program.name']=="The Rachel Maddow Show'Forã\x82Â\xa0 Monday",'program.name'] = 'The Rachel Maddow Show'
full_df.loc[full_df['program.name']=='World Beat','program.name'] = 'Worldbeat'

In [23]:
# final cosmetic changes to make it look more presentable
full_df['program.name'] = full_df['program.name'].str.replace('Cnn','CNN')
full_df['program.name'] = full_df['program.name'].str.replace('Msnbc','MSNBC')
full_df['program.name'] = full_df['program.name'].str.replace("'S","'s")

In [24]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2663266 entries, 0 to 252432
Data columns (total 5 columns):
 #   Column             Dtype         
---  ------             -----         
 0   channel.name       object        
 1   program.name       object        
 2   text               object        
 3   prob_soft_news_us  float64       
 4   full_date          datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 121.9+ MB


## Final cleanup of records

In [25]:
print(full_df.shape)

# removing any outliers, dates that exist in the future of when this was created
full_df.drop(full_df[full_df['full_date'].dt.date >= datetime.date(2022,3,1)].index, axis=0, inplace=True)

# remove any transcript that has less than 100 characters
full_df.drop(full_df[full_df['text'].apply(len) < 100].index, axis=0, inplace =True)

print(full_df.shape)

(2663266, 5)
(2649234, 5)


## Verifying output

In [26]:
sorted(full_df['program.name'].unique().tolist())

['@Thishour With Berman And Michaela',
 'Ac 360 Later',
 'Ahead Of The Curve',
 'All In With Chris Hayes',
 'Amanpour',
 'America`S Voices',
 'American Morning',
 'American Morning: Wake Up Call',
 'American Nightly Scoreboard',
 'Anderson Cooper 360 Degrees',
 'Anthony Bourdain Parts Unknown',
 'Around The World',
 'Asia Tonight',
 'At This Hour',
 'Ballot Bowl 2008',
 'Beck',
 'Biz Asia',
 'Both Sides With Jesse Jackson',
 'Breaking News',
 'Buchanan & Press',
 'Burden Of Proof',
 'Business Traveler',
 'Business Unusual',
 'Business View',
 'CNN 10',
 'CNN America Strikes Back',
 'CNN American Stories',
 'CNN Christmas Around The World',
 'CNN Connie Chung Tonight',
 'CNN Daybreak',
 'CNN Election Center',
 'CNN Evans, Novak, Hunt & Shields',
 'CNN Go',
 'CNN He Said/She Said',
 'CNN Hotspots',
 'CNN In The Money',
 'CNN Insight',
 'CNN International Asia Tonight',
 'CNN International Biz Asia',
 'CNN International Diplomatic Linense',
 'CNN International Inside Africa',
 'CNN Intern

In [27]:
# saving in csv format
full_df.to_csv(PROCESSED_PATH + 'all_news_processed_data.csv')